In [41]:
import ssl
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re

In [20]:
# For ignoring SSL certificate errors  (reused code)

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [25]:
#url = input("Enter URL: ")
url = 'https://www.trulia.com/p/ma/west-roxbury/88-glenellen-rd-west-roxbury-ma-02132--2001575814'

In [27]:
# Use Mozilla access agent

req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()

In [36]:
# Creating a BeautifulSoup object of the html page for easy extraction of data.

soup = BeautifulSoup(webpage, 'html.parser')
html = soup.prettify('utf-8')
property_info = {}

In [37]:
# This code block will get you a one liner description of the listed property

for meta in soup.findAll('meta', attrs={'name': 'description'}):
    try:
        property_info['description'] = meta['content']
        break
    except: 
        pass

property_info['description']

'88 Glenellen Rd, West Roxbury, MA 02132 was recently sold on 08-26-2022 for $445,000. See home details for 88 Glenellen Rd and find similar homes for sale now in West Roxbury, MA on Trulia.'

In [38]:
#Store url 

property_info['link'] = url

property_info['link']

'https://www.trulia.com/p/ma/west-roxbury/88-glenellen-rd-west-roxbury-ma-02132--2001575814'

In [52]:
#Get beds, baths, floorspace

for bed_html in soup.findAll('div', attrs={'data-testid':"home-summary-size-bedrooms"}):
    number_beds = re.findall(r"[0-9]+\s[bB]eds*", str(bed_html))
    if number_beds:
        property_info["beds"] = int(number_beds[0].split(" ")[0])

for bath_html in soup.findAll('div', attrs={'data-testid':"home-summary-size-bathrooms"}):
    number_baths = re.findall(r"[0-9]+\s[bB]aths*", str(bath_html))
    if number_baths:
        property_info["baths"] = int(number_baths[0].split(" ")[0])

for floor_html in soup.findAll('div', attrs={'data-testid':"home-summary-size-floorspace"}):
    floor_footage = re.findall(r"[0-9]+,[0-9]+\ssqft*", str(floor_html))
    property_info["floor_space"] = {}
    if floor_footage:
        property_info["floor_space"]["value"] = int(floor_footage[0].split(" ")[0].replace(",", ""))
        property_info["floor_space"]["unit"] = floor_footage[0].split(" ")[1]

property_info

{'description': '88 Glenellen Rd, West Roxbury, MA 02132 was recently sold on 08-26-2022 for $445,000. See home details for 88 Glenellen Rd and find similar homes for sale now in West Roxbury, MA on Trulia.',
 'link': 'https://www.trulia.com/p/ma/west-roxbury/88-glenellen-rd-west-roxbury-ma-02132--2001575814',
 'beds': 2,
 'baths': 2,
 'floor_space': {'value': 1306, 'unit': 'sqft'}}

In [54]:
#Other known information, don't know how to process yet

for index, sou in enumerate(soup.findAll('span', attrs={"class":"Feature__FeatureListItem-sc-w1mxt5-0 gmLKqq"})):
    print("INDEX", index, ":  ", sou)

INDEX 0 :   <span class="Feature__FeatureListItem-sc-w1mxt5-0 gmLKqq">Basement: Full</span>
INDEX 1 :   <span class="Feature__FeatureListItem-sc-w1mxt5-0 gmLKqq">Number of Rooms: 6</span>
INDEX 2 :   <span class="Feature__FeatureListItem-sc-w1mxt5-0 gmLKqq">Number of Bedrooms: 2</span>
INDEX 3 :   <span class="Feature__FeatureListItem-sc-w1mxt5-0 gmLKqq">Number of Bathrooms: 2</span>
INDEX 4 :   <span class="Feature__FeatureListItem-sc-w1mxt5-0 gmLKqq">Number of Bathrooms (full): 2</span>
INDEX 5 :   <span class="Feature__FeatureListItem-sc-w1mxt5-0 gmLKqq">Living Area: 1306 Square Feet</span>
INDEX 6 :   <span class="Feature__FeatureListItem-sc-w1mxt5-0 gmLKqq">Heating: Forced Air</span>
INDEX 7 :   <span class="Feature__FeatureListItem-sc-w1mxt5-0 gmLKqq">No Cooling</span>
INDEX 8 :   <span class="Feature__FeatureListItem-sc-w1mxt5-0 gmLKqq">Air Conditioning: None</span>
INDEX 9 :   <span class="Feature__FeatureListItem-sc-w1mxt5-0 gmLKqq">Has Heating</span>
INDEX 10 :   <span class=